# Transforming data

In [4]:
import pandas as pd
import numpy as np

import psycopg2 as pg2
from sqlalchemy import create_engine
from os import environ
from sqlalchemy.engine.base import Engine

from raw_to_transformed_data import get_sql_data

pd.set_option("display.max_columns", None)

## Crashes data

In [2]:
crashes_raw_query = """
    SELECT *
    FROM crashes_raw;
    """

In [5]:
df_crashes = get_sql_data(
    db_name="chi-traffic-accidents", query=crashes_raw_query)

In [6]:
df_crashes.head()

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,intersection_related_i,hit_and_run_i,statements_taken_i,photos_taken_i,crash_date_est_i,private_property_i,dooring_i,work_zone_i,work_zone_type,workers_present_i,rd_no,lane_cnt
0,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,2021-08-01T03:42:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",2021-08-01T03:42:00.000,DISREGARDING OTHER TRAFFIC SIGNS,UNABLE TO DETERMINE,4855,S,MORGAN ST,933,2,NO INDICATION OF INJURY,0,0,0,0,0,1,0,3,1,8,41.805384738,-87.650153056,None,None,None,None,None,None,None,None,None,None,None,None
1,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,2021-08-01T02:41:00.000,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",2021-08-01T03:25:00.000,DISREGARDING TRAFFIC SIGNALS,NOT APPLICABLE,99,N,CENTRAL AVE,1522,2,NONINCAPACITATING INJURY,1,0,0,1,0,3,0,2,1,8,41.882148694,-87.764773775,Y,Y,None,None,None,None,None,None,None,None,None,None
2,bba4db7cfef29230aa5d24f6b8912a2539b5d6f481a94e...,2021-08-01T02:26:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",2021-08-01T02:26:00.000,IMPROPER TURNING/NO SIGNAL,IMPROPER TURNING/NO SIGNAL,1537,S,MICHIGAN AVE,132,2,NO INDICATION OF INJURY,0,0,0,0,0,4,0,2,1,8,41.860929115,-87.623859038,None,None,None,None,None,None,None,None,None,None,None,None
3,1e39110c6e01fd2f24e282cd610eb90d6f98a6241ab193...,2021-08-01T02:22:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",2021-08-01T02:25:00.000,UNABLE TO DETERMINE,UNABLE TO DETERMINE,5458,S,WELLS ST,935,2,NONINCAPACITATING INJURY,1,0,0,1,0,1,0,2,1,8,41.794560071,-87.631928475,N,Y,None,None,None,None,None,None,None,None,None,None
4,d20d9db8a88fc227d48392b45380ae57898e5fcdcda7db...,2021-08-01T02:08:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",2021-08-01T02:09:00.000,IMPROPER LANE USAGE,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,151,W,GRAND AVE,1831,3,NONINCAPACITATING INJURY,2,0,0,2,0,3,0,2,1,8,41.89148821,-87.633099826,None,Y,None,None,None,None,None,None,None,None,None,None


In [7]:
df_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527633 entries, 0 to 527632
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   crash_record_id                527633 non-null  object
 1   crash_date                     527633 non-null  object
 2   posted_speed_limit             527633 non-null  object
 3   traffic_control_device         527633 non-null  object
 4   device_condition               527633 non-null  object
 5   weather_condition              527633 non-null  object
 6   lighting_condition             527633 non-null  object
 7   first_crash_type               527633 non-null  object
 8   trafficway_type                527633 non-null  object
 9   alignment                      527633 non-null  object
 10  roadway_surface_cond           527633 non-null  object
 11  road_defect                    527633 non-null  object
 12  report_type                    514502 non-nu

In [31]:
cols_datetime = ["crash_date", "date_police_notified"]
cols_int = ["posted_speed_limit", "lane_cnt", "street_no", 
    "beat_of_occurrence", "num_units", "injuries_total", "injuries_fatal", 
    "injuries_incapacitating", "injuries_non_incapacitating", 
    "injuries_reported_not_evident", "injuries_no_indication", 
    "injuries_unknown", "crash_hour", "crash_day_of_week", "crash_month"]
cols_float = ["latitude", "longitude"]
cols_str = ["crash_record_id", "rd_no", "crash_date_est_i", 
    "traffic_control_device", "device_condition", "weather_condition", 
    "lighting_condition", "first_crash_type", "trafficway_type", "alignment", 
    "roadway_surface_cond", "road_defect", "report_type", "crash_type", 
    "intersection_related_i", "hit_and_run_i", "damage", 
    "prim_contributory_cause", "sec_contributory_cause", "street_direction", 
    "street_name", "photos_taken_i", "statements_taken_i", "dooring_i", 
    "work_zone_i", "work_zone_type", "workers_present_i", "most_severe_injury"]
cols_all = [cols_datetime, cols_int, cols_float, cols_str]
cols_conversions = ["datetime", "float", "integer", "string"]

In [33]:
def convert_df_columns(conversion_type, df, columns):
    if conversion_type not in ("datetime", "float", "integer", "string"):
        raise ValueError(
            "'converstion_type' must be a value of 'datetime', "
            + "'float', 'integer', or 'string'.")
    
    for col in columns:
        if conversion_type == "datetime":
            df[col] = pd.to_datetime(df[col], format="%Y-%m-%dT%H:%M:%S.%f")
        elif conversion_type == "float":
            df[col] = pd.to_numeric(df[col], downcast="float")
        elif conversion_type == "integer":
            df[col] = pd.to_numeric(df[col], downcast="integer")
        elif conversion_type == "string":
            df[col] = df_crashes[col].astype("string")

In [34]:
for conv, cols in zip(cols_conversions, cols_all):
    convert_df_columns(conv, df_crashes, cols)

In [35]:
df_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527633 entries, 0 to 527632
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   crash_record_id                527633 non-null  string        
 1   crash_date                     527633 non-null  datetime64[ns]
 2   posted_speed_limit             527633 non-null  float32       
 3   traffic_control_device         527633 non-null  string        
 4   device_condition               527633 non-null  string        
 5   weather_condition              527633 non-null  string        
 6   lighting_condition             527633 non-null  string        
 7   first_crash_type               527633 non-null  string        
 8   trafficway_type                527633 non-null  string        
 9   alignment                      527633 non-null  string        
 10  roadway_surface_cond           527633 non-null  string        
 11  